In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import mat73

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

# GTS

In [5]:
from models.GTS.graph_learning import GlobalGraphLearning
from models.GTS.DCRNN import DCRNN
# from torch_geometric_temporal.nn.recurrent import DCRNN as dcrnn
from utils.utils import build_edge_idx

from glob import glob
import yaml
from easydict import EasyDict as edict

In [6]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [7]:
node_feas = torch.rand(config.nodes_num, config.node_features, 1000)
edge_index = build_edge_idx(num_nodes=config.nodes_num)

In [8]:
node_feas.shape

torch.Size([30, 1, 1000])

In [9]:
gl = GlobalGraphLearning(config)

In [10]:
adj = gl(node_feas, edge_index)

torch.Size([30, 1, 1000])
torch.Size([30, 1, 1000])
torch.Size([30, 16, 982])
torch.Size([30, 15712])


In [11]:
z_1 = F.gumbel_softmax(adj, tau=0.3, hard=True)
z_1 = torch.transpose(z_1, 0, 1)

In [12]:
edge_ = []

for ii, rel in enumerate(z_1[0]):
    if bool(rel):
        edge_.append(edge_index[:, ii])

adj_matrix = torch.stack(edge_, dim=-1)

In [13]:
config

{'embedding_dim': 64,
 'nodes_num': 30,
 'kernel_size': 10,
 'stride': 1,
 'conv1_dim': 8,
 'conv2_dim': 16,
 'fc_dim': 15712,
 'node_features': 1,
 'diffusion_k': 3,
 'num_layer': 1,
 'tau': 0.3,
 'output_dim': 1,
 'encoder_length': 70,
 'decoder_length': 30}

In [14]:
encoder_dcrnn = DCRNN(config)
decoder_dcrnn = DCRNN(config)

In [15]:
inputs = node_feas.reshape(-1,30,1,100)
inputs.shape
# batch, node_num, node_feature, total time step

torch.Size([10, 30, 1, 100])

In [16]:
encoder_hidden_state = encoder_dcrnn(inputs[0].reshape(config.nodes_num,-1), adj_matrix)

torch.Size([30, 100])
Initial Hidden State Dim: torch.Size([30, 64])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (30x164 and 128x64)

In [ ]:
from models.GTS.DCGRU import DCRNN

In [ ]:
dcrnn = DCRNN(64, 64, 3)

In [ ]:
dcrnn(inputs[0][:,:,0], adj_matrix)